In [51]:
using PrettyTables

function RK4(f, x0, y0, N, h)
    #Use a 4th order Runge-Kutta method to solve y'=f(x,y), with y(x0)=y0 using N steps of size h.
    #Returns an array of (x_i, w_i) pairs.
    approx = [x0 y0]
    x = x0
    w = y0
    for i in 1:N
        k1 = f(x, w)
        k2 = f(x + h / 2, w + h * k1 / 2)
        k3 = f(x + h / 2, w + h * k2 / 2)
        k4 = f(x + h, w + h * k3)
        w = w + h / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
        x += h
        approx = vcat(approx, [x w]) #Add a new row to the matrix of approximation points
    end
    return approx
end

function Newton(f, df, x0, eps, N) #Performs Newton's method with a maximum of N iterations on the function f, starting from x0.
    i = 1
    while i <= N
        x = x0 - f(x0) / df(x0) #Calculate next iteration with Newton's method
        if abs(x - x0) < eps #Stopping criteria satisfied, root found
            return x
        else
            x0 = x #Updates x0 for next iteration
        end
        i = i + 1
    end
    println("Newton's method failed to converge") #Performed N iterations without converging
end

function AM2_Explicit(f, x0, y0, N, h)
    #Two step Adams-Moulton method.
    x1 = x0 + h
    x2 = x1 + h
    w0 = y0
    RK_approx = RK4(f, x0, y0, 1, h)
    w1 = RK_approx[2, 2]
    approx = [x0 w0; x1 w1]
    for i in 1:N-1 #We already did one step with RK4
        w2 = (12 * w1 + h*(8*x1^2*w1-x0^2*w0))/(12-5*h*x2^2)
        approx = vcat(approx, [x2 w2])
        #Update starting values
        x0 = x1
        x1 = x2
        x2 += h
        w0 = w1
        w1 = w2
    end
    return approx
end

#Below is a general Adams-Bashforth methods.  Each uses RK4 to derive its starting values.
function AB2(f, x0, y0, N, h)
    x1 = x0 + h
    w0 = y0
    RK_approx = RK4(f, x0, y0, 1, h)
    w1 = RK_approx[2, 2]
    approx = [x0 w0; x1 w1]
    for i in 1:N-1
        x2 = x1 + h
        w2 = w1 + h * (1.5 * f(x1, w1) - .5 * f(x0, w0))
        approx = vcat(approx, [x2 w2])
        x0 = x1
        x1 = x2
        w0 = w1
        w1 = w2        
    end
    return approx
end

function AB3(f, x0, y0, N, h)
    x1 = x0 + h
    x2 = x1 + h
    w0 = y0
    RK_approx = RK4(f, x0, y0, 2, h)
    w1 = RK_approx[2, 2]
    w2 = RK_approx[3, 2]
    approx = [x0 w0; x1 w1; x2 w2]
    for i in 1:N-2
        x3 = x2 + h
        w3 = w2 + h / 12 * (23 * f(x2, w2) - 16 * f(x1, w1) + 5 * f(x0, w0))
        approx = vcat(approx, [x3 w3])
        x0 = x1
        x1 = x2
        x2 = x3
        w0 = w1
        w1 = w2
        w2 = w3
    end
    return approx
end

function AB4(f, x0, y0, N, h)
    x1 = x0 + h
    x2 = x1 + h
    x3 = x2 + h
    w0 = y0
    RK_approx = RK4(f, x0, y0, 3, h)
    w1 = RK_approx[2, 2]
    w2 = RK_approx[3, 2]
    w3 = RK_approx[4, 2]
    approx = [x0 w0; x1 w1; x2 w2; x3 w3]
    for i in 1:N-3
        x4 = x3 + h
        w4 = w3 + h / 24 * (55 * f(x3, w3) - 59 * f(x2, w2) + 37 * f(x1, w1) - 9 * f(x0, w0))
        approx = vcat(approx, [x4 w4])
        x0 = x1
        x1 = x2
        x2 = x3
        x3 = x4
        w0 = w1
        w1 = w2
        w2 = w3
        w3 = w4
    end
    return approx
end

#Below are some general Adams-Moulton methods.  Each uses RK4 to determine starting values and Newton's method to solve the implicit equation.
function AM2(f, fy, x0, y0, N, h)
    x1 = x0 + h
    x2 = x1 + h
    w0 = y0
    RK_approx = RK4(f, x0, y0, 1, h)
    w1 = RK_approx[2, 2]
    approx = [x0 w0; x1 w1]
    for i in 1:N-1
        g(w2) = w2 - w1 - h / 12 * (5 * f(x2, w2) + 8 * f(x1, w1) - f(x0, w0)) #Non linear function whose root is w2
        dg(w2) = 1 - 5 * h / 12 * fy(x2, w2) #Derivative of g for Newton's method
        w2 = Newton(g, dg, w1, h^3, 10000)
        approx = vcat(approx, [x2 w2])
        #Update starting values
        x0 = x1
        x1 = x2
        x2 += h
        w0 = w1
        w1 = w2
    end
    return approx
end

function AM3(f, fy, x0, y0, N, h)
    x1 = x0 + h
    x2 = x1 + h
    x3 = x2 + h
    w0 = y0
    RK_approx = RK4(f, x0, y0, 2, h)
    w1 = RK_approx[2, 2]
    w2 = RK_approx[3, 2]
    approx = [x0 w0; x1 w1; x2 w2]
    for i in 1:N-2
        g(w3) = w3 - w2 - h / 24 * (9 * f(x3, w3) + 19 * f(x2, w2) - 5 * f(x1, w1) + f(x0, w0))
        dg(w3) = 1 - 9 * h / 24 * fy(x3, w3) #Derivative of g for Newton's method
        w3 = Newton(g, dg, w2, h^4, 10000)
        approx = vcat(approx, [x3 w3])
        #Update starting values
        x0 = x1
        x1 = x2
        x2 = x3
        x3 += h
        w0 = w1
        w1 = w2
        w2 = w3
    end
    return approx
end


AM3 (generic function with 1 method)

In [39]:
#Example using a two step Adams-Moulton method to solve y' = x^2y, y(0) = 1.
#The method is thrid order.
f(x, y) = x^2*y
exact(x) = exp(x^3/3)
a = 0
b = 1
y0 = 1
errors = []
for i in 1:4
    N = 10^i
    h = (b - a) / N
    approx = AM2(f, a, y0, N, h)
    y = approx[N + 1, 2]
    e = abs(y - exact(b))
    if i == 1
        errors = [h e]
    else
        errors = vcat(errors, [h e])
    end
end
pretty_table(errors, column_labels = ["h", "Error"])

┌────────┬─────────────┐
│      h │       Error │
├────────┼─────────────┤
│    0.1 │ 0.000436077 │
│   0.01 │  5.02284e-7 │
│  0.001 │ 5.09609e-10 │
│ 0.0001 │ 4.32987e-13 │
└────────┴─────────────┘


In [56]:
#Example comparing Adams-Bashforth and Adams-Moulton methods to solve y' = y^-2, y(0) = 1.
f(x, y) = 1 / y^2
fy(x, y) = -2 / y^3
exact(x) = (3 * x + 1)^(1 / 3)
a = 0
b = 1
y0 = 1
errors = []
for i in 1:4
    N = 10^i
    h = (b - a) / N
    approx = AB2(f, a, y0, N, h)
    y_ab2 = approx[N + 1, 2]
    e_ab2 = abs(y_ab2 - exact(b))
    println("Adams-Bashforth Time: ")
    @time begin #We'll compare times for the comparable AB and AM methods
    approx = AB3(f, a, y0, N, h)
    y_ab3 = approx[N + 1, 2]
    e_ab3 = abs(y_ab3 - exact(b))
    approx = AB4(f, a, y0, N, h)
    y_ab4 = approx[N + 1, 2]
    e_ab4 = abs(y_ab4 - exact(b))
    end
    println("Adams-Moulton Time: ")
    @time begin
    approx = AM2(f, fy, a, y0, N, h)
    y_am2 = approx[N + 1, 2]
    e_am2 = abs(y_am2 - exact(b))
    approx = AM3(f, fy, a, y0, N, h)
    y_am3 = approx[N + 1, 2]
    e_am3 = abs(y_am3 - exact(b))
    end
    if i == 1
        errors = [h e_ab2 e_ab3 e_am2 e_ab4 e_am3]
    else
        errors = vcat(errors, [h e_ab2 e_ab3 e_am2 e_ab4 e_am3])
    end
end
pretty_table(errors, column_labels = ["h", "AB2", "AB3", "AM2", "AB4", "AM3"])

Adams-Bashforth Time: 
  0.089877 seconds (28.69 k allocations: 1.929 MiB, 99.91% compilation time: 100% of which was recompilation)
Adams-Moulton Time: 
  0.055395 seconds (12.42 k allocations: 819.484 KiB, 99.47% compilation time: 81% of which was recompilation)
Adams-Bashforth Time: 
  0.000082 seconds (448 allocations: 197.531 KiB)
Adams-Moulton Time: 
  0.000794 seconds (11.92 k allocations: 425.797 KiB)
Adams-Bashforth Time: 
  0.004710 seconds (4.05 k allocations: 15.738 MiB, 37.42% gc time)
Adams-Moulton Time: 
  0.007208 seconds (120.42 k allocations: 18.003 MiB)
Adams-Bashforth Time: 
  0.403724 seconds (58.01 k allocations: 1.494 GiB, 13.62% gc time)
Adams-Moulton Time: 
  0.458120 seconds (1.22 M allocations: 1.516 GiB, 12.57% gc time)
┌────────┬────────────┬─────────────┬─────────────┬─────────────┬─────────────┐
│      h │        AB2 │         AB3 │         AM2 │         AB4 │         AM3 │
├────────┼────────────┼─────────────┼─────────────┼─────────────┼─────────────┤
│ 